<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5BConf%5D_physio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00


In [2]:
pip install bioread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 938.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5460d88935b718ad3b9560648dc97e97594216aa4b2ea3a7ba5610fc06c76552
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
### preparation
import neurokit2 as nk
import bioread as br
import os
import pandas as pd
import numpy as np

### set path
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Res/Conformity/data')
datapath = r'/content/drive/MyDrive/Res/Conformity/data'
savepath = r'/content/drive/MyDrive/Res/Conformity/analyses'

Mounted at /content/drive


In [4]:
### import and set raw data: range 설정 필! 한 파일에 두번 돌리지 말 것.

tasklist = ['HRV', 'HCT', 'BlockI', 'BlockII']

for i in range(32, 33):

  txtpath = datapath+f'/sub{i}.txt'

  with open(txtpath, "r") as file:
      lines = file.readlines()
  with open(txtpath, "w") as file:
      file.writelines(lines[23:])

  data = pd.read_csv(txtpath, delimiter='\t', low_memory=False)
  data = data.iloc[1:]; data = data.iloc[:, :-1]

  signal = []
  for j in range(1, len(data)):
    if data['CH25'][j] == '0' and data['CH25'][j+1] == '5':
      signal.append(j+1)

  for j in range(4):
    slicedf = data.iloc[signal[2*j]:signal[2*j+1]]
    savefile = datapath+f'/sub{i}_{tasklist[j]}.csv'
    slicedf.to_csv(savefile, index=True)


In [5]:
### HRV: range 설정 필!

hrvdf = pd.DataFrame()

for i in range(32,33):

  ## SubID
  SubID = str(i)
  hrvdf.loc[0, 1] = SubID
  hrvdf.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvnonlin], axis=1, ignore_index=False)

for i in range(32,33):
  hrvdf1 = pd.DataFrame()

  ## SubID
  SubID = str(i)
  hrvdf1.loc[0, 1] = SubID
  hrvdf1.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvnonlin], axis=1, ignore_index=False)
  hrvdf = pd.concat([hrvdf, hrvdf1], ignore_index=True)

savefile = savepath+f'/HRVDataIII_32.xlsx'
hrvdf.to_excel(savefile, index=False)

hrvdf

,SubID,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,32,614.179795,39.473901,17.088638,34.813986,14.767691,37.201085,NaN,NaN,16.778345,...,1.187017,7.776228,0.825244,1.566134,1.357489,2.181597,1.611311,1.652194,2.98177,0.62944
1,32,614.179795,39.473901,17.088638,34.813986,14.767691,37.201085,NaN,NaN,16.778345,...,1.187017,7.776228,0.825244,1.566134,1.357489,2.181597,1.611311,1.652194,2.98177,0.62944


In [8]:
### HCT: range 설정 필!

hctdf = pd.DataFrame()
hctdf.loc[1, 1] = '1'  #여기도 수정 필!
hctdf.columns = ['SubID']

for i in range(32,33):

  ## SubID
  SubID = str(i)
  hctdf.loc[i, 'SubID'] = SubID

  ## Extract basic features
  hctfile = datapath+f'/sub{SubID}_HCT.csv'
  data = pd.read_csv(hctfile)

  processed_df, info = nk.bio_process(ecg=data['CH1'], rsp=data['CH3'], sampling_rate=2000)

  ## Set epochs
  hctpath = datapath+f'/sub{i}_HCT.csv'
  data = pd.read_csv(hctpath, low_memory=False)

  startsignal = []; endsignal = []
  for j in range(len(data)-1):
    if data['CH21'][j] == 0 and data['CH21'][j+1] == 5:
      startsignal.append(j+1)
    elif data['CH21'][j] == 5 and data['CH21'][j+1] == 0:
      endsignal.append(j+1)

  ## count R-peaks in each epoch
  hctdf1 = pd.DataFrame()
  for j in range(6):
    slicedf = processed_df.iloc[startsignal[2*j]:endsignal[2*j]]
    rpeaks = slicedf['ECG_R_Peaks'].sum()
    hctdf1.loc[j+1, 1] = rpeaks

  ## calculate indices for interoceptive sensitivity
  rawcardf = pd.read_excel(datapath+f'/HCT/Cardiac-{i}.xls', header=1)
  rawcardf = rawcardf[['HC.RESP','HCCR.Slider1.Value']].iloc[2:8]
  rawcardf.index = range(1,7)
  hctdf1 = pd.concat([hctdf1, rawcardf], axis=1, ignore_index=False)
  hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1[1]
  hctdf1['alteIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])*2/(hctdf1['HC.RESP']+hctdf1[1])
  hctdf1['alteCheck'] = hctdf1['HC.RESP'] - hctdf1[1]*2
  if (hctdf1['alteCheck'] > 0).any():
    print(f'Need alteIAcc analyses due to sub{i}')
  hctdf.loc[i, 'stanIAcc'] = hctdf1['stanIAcc'].mean()
  hctdf.loc[i, 'alteIAcc'] = hctdf1['alteIAcc'].mean()
  hctdf.loc[i, 'IS'] = hctdf1['HCCR.Slider1.Value'].mean()
  hctdf.loc[i, 'stanIAw'] = hctdf1['stanIAcc'].corr(hctdf1['HCCR.Slider1.Value'])
  hctdf.loc[i, 'alteIAw'] = hctdf1['alteIAcc'].corr(hctdf1['HCCR.Slider1.Value'])

hctdf = hctdf[['SubID', 'stanIAcc', 'alteIAcc', 'IS', 'stanIAw', 'alteIAw']]
savefile = savepath+f'/HCTData_PilotIII_32_2.xlsx'
hctdf.to_excel(savefile, index=False)

hctdf

,SubID,stanIAcc,alteIAcc,IS,stanIAw,alteIAw
1,1,NaN,NaN,NaN,NaN,NaN
32,32,0.391091,0.077209,2.833333,0.681513,0.664862


In [ ]:
hctdf1

,1,HC.RESP,HCCR.Slider1.Value,stanIAcc,alteIAcc,alteCheck
1,81.0,29.0,4.0,-0.793103,0.054545,-133.0
2,91.0,23.0,3.0,-1.956522,-0.192982,-159.0
3,75.0,37.0,7.0,-0.027027,0.321429,-113.0
4,66.0,34.0,9.0,0.058824,0.360000,-98.0
5,57.0,21.0,2.0,-0.714286,0.076923,-93.0
6,49.0,14.0,3.0,-1.500000,-0.111111,-84.0
